In [1]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime
from nltk.stem import WordNetLemmatizer
API_KEY = ""

In [34]:
llm = ChatOpenAI(api_key=API_KEY)
output_parser = StrOutputParser()

def chat(template,message):
    prompt = ChatPromptTemplate.from_messages( [
    ("system", template),
    ("user", "{input}")
    ])
    chain = prompt | llm | output_parser

    response = chain.invoke({"input": message})
    return response




NameError: name 'API_KEY' is not defined

In [3]:
intent_find_template = f"""
You are a Natural Language Understanding  agent. Given a query from user you need to find out the intent of user. Some examples are given below.

1) GetTransactions
    ie1: list transaction I have done this week.
    response1 : "intent" : "GetTransactions"
    ie2: list transaction I have done during february.
    response2 : "intent" : "GetTransactions" 

2) GetTransactionsAmount 
    ie1: How much did I spent today?
    reponse1: "intent" : "GetTransactionAmount", "type"  :  "credit" 
    ie2: What were my total ex dpenses during last year?
    response2: "intent" : "GetTransactionAmount", "type"  :  "credit"
    ie3: How much amount I deposited during last month of 2023?
    response3 : "intent"  :  "GetTransactionAmount" , "type" : "debit"

3) GetCategoryTransactions : categoryname
    ie1: List my transactions I have done on groceries?
    response1 : "intent"  :  "GetCategoryTransactions " , "categoty" : " groceries"
    ie2: List my transactions for Starbucks?
    response2 : "intent"  :  "GetCategoryTransactions", "category" : "Starbucks"
        
4) GetCategoryTransactionsAmount: categoryname
    ie1: How much I have spent on shops so far?
    response1: "intent"  :  "GetCategoryTransactionsAmount" , "type" : "credit", "category" : "shops"
    ie2: How much I have spent on Starbucks so far?
    response2 : "intent"  :  "GetCategoryTransactionsAmount " , "type" : "credit ", "category" : "Starbucks"
    ie3: How much I have deposited via zelle this month.
    reponse3: "intent"  :  "GetCategoryTransactionsAmount " , "type" : " debit ", "category" : " zelle"
   ie 4: How much amount I have deposited using cash app?
    response4: "intent"  :  "GetCategoryTransactionsAmount  " , "type" : "debit , "category" : " cash app"

------------------------------------------------------------------------------------
Now reply with intent for following query. A cash or any kind of  inflow is considered as debit while cash or other kind of outflow credit. Output result in json as mentioned in examples

Query: 
"""


In [4]:
date_find_prompt = f"""
You are a very precise date finding agent. Some examples of your work are as follow. You generate a json as output:
    Today is {datetime.now()} and Day is {datetime.now().strftime("%A")}

    Query 1: How much did I spend on Starbuck yesterday.
    response: "date:2024-04-24"

    Query: List transactions I have done during last week.
    response: "date:2024-04-15, 2024-04-21"

    Query: How much did I spent this month.
    response: "date: 2024-04-01 , 2024-04-25"

    Query: How much did I spend last year
    response: "date:2023-01-01, 2023-12-30"

    Note: 1) last year means from from 1 jan 2023 to 31 Dec of 2023, similarly last 2 years 
          means 2022-01-01, 2023-01-01. Last two months means 2024-02-01,2024-03-31
          
          2) A week ends on Sunday, so during this week mean starting from Monday Till Sunday 
            which  will be 2024-04-22, 2024-04-26.
            
          3) If first Day of a month starts from Wednesday then first week means from Wednesday till 
            Sunday
            
          4) last week or last month excludes current week or month. similarly last two weeks doesn't include current week.
    -------------------------------------------------------------------------------
    Now answer the following query: Only output json with keys start_date,end_date if there is interval, otherwise with date key. No text or comments. If there is no date for query just leave the field empty
    Query: """

In [30]:
class DataFetcher:
    def __init__(self,client_id = 2044309):
        data = pd.read_csv("Processed_Data.csv")
        data['txn_date'] = pd.to_datetime(data["txn_date"])
        self.data = data[data["clnt_id"] == client_id]
        self.categories = data["lemm_cat"].unique()
        self.merchants = data["lemm_merchant"].unique()
        self.lemmatizer = WordNetLemmatizer()
    
    def _clean_listing(self,listing):
        listing = listing.drop(['clnt_id','bank_id','acc_id','txn_id','lemm_merchant','lemm_cat'],axis=1)
        return listing
    
    def _get_transactions(self,dates):
        dates = [date for date in dates.values()]
        if len(dates)==1:
            listing = self.data[self.data["txn_date"] == pd.to_datetime(dates[0])]
        else:
            listing = self.data[(self.data["txn_date"] > pd.to_datetime(dates[0])) & (self.data["txn_date"] < pd.to_datetime(dates[1]))]
        return listing

    def GetTransactions(self,dates):
        listing = self._get_transactions(dates)
        listing = self._clean_listing(listing)
        dates = [date for date in dates.values()]
        
        if len(dates)==1:
            return f"Following are your Transactions for the date : {dates[0]}\n {listing}"
        else:
            return f"Following are your Transactions between : {dates[0]} and {dates[1]}.\n {listing}"
    
   
    
    def GetTransactionsAmount(self,trn_type,dates):
        dates = [date for date in dates.values()]
        if trn_type == "credit":
            if len(dates)==1:
                listing = self.data[self.data["txn_date"] == pd.to_datetime(dates[0])]
                return f"You spent {str(listing[listing['amt']<0]['amt'].sum()).lstrip('-')} on {dates[0]}\n "

            else:
                listing = self.data[(self.data["txn_date"] > pd.to_datetime(dates[0])) & (self.data["txn_date"] < pd.to_datetime(dates[1]))]
                return f"You spent {str(listing[listing['amt']<0]['amt'].sum()).lstrip('-')} between {dates[0]} and {dates[1]}\n "
        else:
            if len(dates)==1:
                listing = self.data[self.data["txn_date"] == pd.to_datetime(dates[0])]
                return f"You spent {str(listing[listing['amt']>0]['amt'].sum()).lstrip('-')} on {dates[0]}\n "

            else:
                listing = self.data[(self.data["txn_date"] > pd.to_datetime(dates[0])) & (self.data["txn_date"] < pd.to_datetime(dates[1]))]
                return f"You spent {str(listing[listing['amt']>0]['amt'].sum()).lstrip('-')} between {dates[0]} and {dates[1]}\n "
                
    
    def GetCategoryTransactions(self,merchant,dates=None):
        filtered_data = self.data
        if self._is_valid_date(dates):
            filtered_data = self._get_transactions(dates)
        merchant = self.lemmatizer.lemmatize(str(merchant).lower())
        listing = None
        if merchant in self.merchants:
            listing = filtered_data[filtered_data['lemm_merchant'] == merchant]
        elif merchant in self.categories:
            listing = filtered_data[filtered_data['lemm_cat'] == merchant]
        if listing is not None and len(listing) > 0:
            listing = self._clean_listing(listing)
            return f"Following are your transactions for {merchant}.\n{listing}"
        else:
            return "No transaction found"
    
    def GetCategoryTransactionsAmount(self,trn_type,merchant,dates=None):
        filtered_data = self.data
        if self._is_valid_date(dates):
            filtered_data = self._get_transactions(dates)
        
        amount = 0
        listing = None
        merchant = self.lemmatizer.lemmatize(str(merchant).lower())
        if merchant in self.merchants:
            listing = filtered_data[filtered_data['lemm_merchant'] == merchant]
        elif merchant in self.categories:
            listing = filtered_data[filtered_data['lemm_cat'] == merchant]
        if listing is not None and len(listing) > 0:
            if trn_type == "credit":
                listing = listing[listing["amt"]<0]
            else:
                listing = listing[listing["amt"]>0]
            amount = listing["amt"].sum()
        return f"You have spent {amount} for {merchant}"
    
    def _is_valid_date(self,dates):
        if dates:
            dates = [date for date in dates.values() if date != ""]
            if dates:
                return True
        return False


In [32]:
import json
result = []
queries = ["How much did I spend on Empowerment in during of August 2023?",
          "What transactions I have made on RoarCurrency",
          "List transactions I have made during July 2023",
           "What were my expenses during 2023?",
            "I need to see transactions I have done on 2023-09-01"
          ]
          
for query in queries:  
    intent,date = chat(intent_find_template,query),chat(date_find_prompt,query)
    print(intent,date)
    result.append([json.loads(intent),json.loads(date)])
    

{
  "intent": "GetCategoryTransactionsAmount",
  "type": "credit",
  "category": "Empowerment"
} {
    "start_date": "2023-08-01",
    "end_date": "2023-08-31"
}
{
    "intent": "GetCategoryTransactions",
    "category": "RoarCurrency"
} {}
{
    "intent": "GetTransactions",
    "month": "July",
    "year": "2023"
} {
    "start_date": "2023-07-01",
    "end_date": "2023-07-31"
}
{
    "intent": "GetTransactionsAmount",
    "type": "credit"
} {
    "start_date": "2023-01-01",
    "end_date": "2023-12-31"
}
{
    "intent": "GetTransactions",
    "date": "2023-09-01"
} {
    "date": "2023-09-01"
}


In [125]:
result = [
    [{'intent': 'GetCategoryTransactionsAmount',
   'type': 'credit',
   'category': 'Empower'},
  {'start_date': '2023-08-01', 'end_date': '2023-08-31'}],
 [{'intent': 'GetCategoryTransactions', 'category': 'MoneyLion'},
  {'date': ''}],
 [{'intent': 'GetTransactions'},
  {'start_date': '2023-07-01', 'end_date': '2023-07-31'}],
 [{'intent': 'GetTransactionsAmount', 'type': 'credit'},
  {'start_date': '2023-01-01', 'end_date': '2023-12-31'}],
 [{'intent': 'GetTransactions'}, {'date': '2023-09-01'}]]

In [28]:
def get_function(class_inst,method_name):
    method = None
    try:
        method = getattr(class_inst, method_name)
    except AttributeError:
        raise NotImplementedError("Class `{}` does not implement `{}`".format(class_inst.__class__.__name__, method_name))

    return method

In [33]:
data_fetcher = DataFetcher()

for definition,date in result:
    func = definition["intent"]
    method = get_function(data_fetcher,func)
    category = definition.get("category",None)
        
    type = definition.get("type",None)
    if category:
        if type:    
            answer = method(type,category,date)
        else:
            answer = method(category,date)
    else:
        if type:
            answer = method(type,date)
        else:
            answer = method(date)
    print(answer)
    

You have spent -7966.933999999999 for empowerment
Following are your transactions for roarcurrency.
        Unnamed: 0   txn_date     amt       cat      merchant
23580        23580 2023-08-15 -13.598   Service  RoarCurrency
23789        23789 2023-08-29 -14.598   Service  RoarCurrency
24217        24217 2023-07-05 -13.598   Service  RoarCurrency
24219        24219 2023-08-01 -14.596   Service  RoarCurrency
24344        24344 2023-07-20 -14.398   Service  RoarCurrency
...            ...        ...     ...       ...           ...
235446      235446 2023-09-22  -0.920  Transfer  RoarCurrency
235479      235479 2023-09-15  -0.920  Transfer  RoarCurrency
235484      235484 2023-07-07  -0.920  Transfer  RoarCurrency
235487      235487 2023-09-01  -0.920  Transfer  RoarCurrency
235602      235602 2023-06-16  -1.998  Transfer  RoarCurrency

[2105 rows x 5 columns]
Following are your Transactions between : 2023-07-01 and 2023-07-31.
         Unnamed: 0            txn_date     amt    cat        